# LLM ModelTest #2
#### *ibm-granite/granite-embedding-278m-multilingual*

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API KEY 정보로드
load_dotenv()
GEMINI_API_KEY = os.getenv('API_KEY_GEMINI')

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("dx_project")

LangSmith 추적을 시작합니다.
[프로젝트명]
dx_project


In [33]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS

from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd
import re
import json
# 랭체인 환경 설정
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
#from langchain_core.prompts import PromptTemplate

from langchain.prompts import ChatPromptTemplate
# from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

In [5]:
### 01. CSV 파일에서 문서 로드 ###
loader = CSVLoader('../data/movie_4000_preprocessed.csv', encoding='utf8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

### 02. pandas로 데이터프레임 칼럼명 가져오기
csv_path = '../data/movie_4000_preprocessed.csv'
df = pd.read_csv(csv_path, encoding='utf8')
colnames = df.columns

문서의 수: 3901


In [6]:
### 03. 메타데이터 추가 ###
docs = []
for _, row in df.iterrows():
  # 필요한 메타데이터 설정
  metadata = {
    'title': row['movie_title'],
    'genre': row['genre']
  }
  # 각 행의 데이터를 문서로 변환
  doc = Document(
    page_content=str(row.to_dict()),
    metadata=metadata
  )
  docs.append(doc)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[100].metadata)

문서의 수: 3901
[메타데이터 예시]
 {'title': '아름답다', 'genre': '드라마'}


In [7]:
### 04. 데이터 청크 나누기 ###
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1090, chunk_overlap=0
)
splits = text_splitter.split_documents(docs)
print("split된 문서의 수:", len(splits))

split된 문서의 수: 3901


In [8]:
### 05. 임베딩 모델 생성
# https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual
embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')

C:\Users\USER\AppData\Local\Temp\ipykernel_11556\3636801640.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='ibm-granite/granite-embedding-278m-multilingual')
c:\workspaces\proj\forllm\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [39]:
# Chroma 벡터스토어 로드
vectorstore = Chroma(persist_directory="../data/movie_4000_vectorstore_2", embedding_function=embeddings)
# Fassis 벡터스토어 로드
vectorstore_faiss = FAISS.load_local('../data/movies_vectorstore_faiss_1500', embeddings, allow_dangerous_deserialization=True)

In [10]:
# 1. 사용자 입력값 유형 분류용 프롬프트
classification_template = """사용자의 입력을 다음 세 가지 범주 중 하나로 분류하세요:
1️⃣ **"정보검색"**: 특정 영화, 드라마, 배우, 감독, 러닝타임, 개봉 연도, 수상 내역, 필모그래피 등 **사실적인 정보를 찾는 질문**
   - 기대되는 응답 예시: 배우가 출연한 드라마/영화 목록, 특정 연도의 개봉작 리스트 등
2️⃣ **"추천요청"**: 특정 장르, 배우, 테마(예: 좀비, 시간여행), 감성(예: 힐링, 긴장감) 등에 대한 **추천을 요청하는 질문**
   - 기대되는 응답 예시: 특정 조건을 만족하는 영화/드라마 추천
3️⃣ **"일반대화"**: 서비스와 무관한 일반적인 대화 (예: 날씨, AI 관련 질문, 잡담)

질문과 관련된 주요 키워드를 반환하세요.

#### **예시 형식**
{{
  "type": "정보검색",
  "keywords": ["한효주", "드라마", "출연"]
}}

<question>
{question}
</question>
"""
prompt = ChatPromptTemplate.from_template(classification_template)

llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0)

In [11]:
# langchain 체인 구성
classification_chain = (
  prompt
  | llm
  | StrOutputParser()
)

In [12]:
# 예상 정보검색
print(classification_chain.invoke({"question": "한효주가 나오는 드라마 알려줘."}))
print(classification_chain.invoke({"question": "스파이더맨 영화를 보고싶어"}))

# 추천 요청
print(classification_chain.invoke({"question": "코미디면서 호러 장르의 영화를 추천해줘"}))

# 일반대화화
print(classification_chain.invoke({"question": "심심해"}))
print(classification_chain.invoke({"question": "오늘은 우울한걸"}))


{
  "type": "정보검색",
  "keywords": ["한효주", "드라마", "출연"]
}
{
  "type": "추천요청",
  "keywords": ["스파이더맨", "영화", "추천"]
}
{
  "type": "추천요청",
  "keywords": ["코미디", "호러", "영화", "추천"]
}
{
  "type": "일반대화",
  "keywords": ["심심해"]
}
{
  "type": "일반대화",
  "keywords": ["우울", "감정"]
}


In [13]:
# JSON 결과를 올바르게 파싱하는 함수
def preprocess_classification_result(question: str):
    classification_result = classification_chain.invoke({"question": question})

    # JSON 문자열에서 ```json``` 제거 (정규식 활용)
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', classification_result, re.DOTALL)
    if json_match:
        clean_json_str = json_match.group(1)                       # 중괄호 {} 내부만 추출
    else:
        clean_json_str = classification_result                     # ```json``` 태그가 없을 경우 그대로 사용

    # JSON 문자열을 dict 자료형으로 변환
    try:
        classification_data = json.loads(clean_json_str)
        type_value = classification_data.get("type", "일반대화")   # 기본값을 '일반대화'로 설정
        keywords = classification_data.get("keywords", [])         # 기본값을 빈 리스트로 설정

    except json.JSONDecodeError as e:
        print(f"⚠️ JSONDecodeError: {e}")  # JSON 변환 오류 확인
        type_value = "일반대화"           # JSON 변환 오류 시 기본값 설정
        keywords = []
    
    return {"type": type_value, "keywords": keywords}

In [14]:
# default_chain 생성 (사용자의 의미없는 입력값에 대해 정해진 답변을 할 때)
# 프롬프트 템플릿 설정
default_template = """
"You are a chatbot that must always respond with '🐶: 멍멍!'.
No matter what question the user asks, always reply with '🐶: 멍멍!'"

[사용자 입력과 분류 결과]:
{classification_result}
"""
default_prompt = ChatPromptTemplate.from_template(default_template)

default_llm = ChatOpenAI(
  model="gpt-4o-mini",
  temperature=0)

# langchain 체인 구성
default_chain = (
  {"classification_result": RunnablePassthrough()}
  | default_prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | default_llm            # llm이 원하는 답변을 만듦
  | StrOutputParser()
)

# langchain router
##  정보검색

In [29]:
# 검색기 생성
# mmr 중복 피하기, 문서의관련성과 차별성 고려, 
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",   
    search_kwargs={"score_threshold": 0.5}
)

info_template = """
사용자가 영화나 드라마에 대한 정보를 검색하고 있습니다.
다음 사용자 질문과 관련된 **가장 적절한 문서(컨텐츠)를 벡터스토어에서 검색**한 후, 
검색 결과에서 최대 5개 문서에 대해서 출력하세요.

### **예시 응답 형식**
{{
  "title": "더 이퀄라이저",
  "genre": ["SF", "스릴러"]

  "title": "더 이퀄라이저 2",
  "genre": ["SF", "스릴러"]

  "title": "더 이퀄라이저 3",
  "genre": ["SF", "스릴러"]

}}

[사용자 입력과 분류 결과]:
{classification_result}

[Context]: 
{retrieved_context} 
Answer:

"""
info_prompt = ChatPromptTemplate.from_template(info_template)
info_chain = (
    {
      "classification_result": RunnablePassthrough(),
      "retrieved_context": retriever
    }
    |info_prompt
    |llm
    )

In [31]:
info_chain.invoke("더 이퀄라이저 시리즈가 몇개야 내놔")

AIMessage(content='{\n  "documents": [\n    {\n      "title": "더 이퀄라이저",\n      "genre": ["액션/어드벤쳐"]\n    },\n    {\n      "title": "더 이퀄라이저 2",\n      "genre": ["액션/어드벤쳐"]\n    },\n    {\n      "title": "더 이퀄라이저 3",\n      "genre": ["액션/어드벤쳐"]\n    }\n  ]\n}', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 965, 'total_tokens': 1066, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-1052c05f-9a42-4584-bfd7-3005c4cb9ffe-0', usage_metadata={'input_tokens': 965, 'output_tokens': 101, 'total_tokens': 1066, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 테디

In [35]:
def route(info):
    # 주제에 "정보검색"이 포함되어 있는 경우
    if "정보검색" in info["topic"].lower():
        return info_chain
    # 주제에 "추천요청"이 포함되어 있는 경우
    elif "추천요청" in info["topic"].lower():
        return recommend_chain
    # 일반대화화
    else:
        return general_chain
    
# from operator import itemgetter
# from langchain_core.runnables import RunnableBranch

# branch = RunnableBranch(
#     (lambda x: "정보검색" in x["topic"].lower(), info_chain),
#     (lambda x: "추천요청" in x["topic"].lower(), recommend_chain),
#     general_chain,
# )
# full_chain = (
#     {"topic": rag_chain, "question": itemgetter("question")} | branch | StrOutputParser()
# )

In [36]:
full_chain = (
    {"topic": rag_chain, "question": itemgetter("question")}
    | RunnableLambda(
        route
    )
    | StrOutputParser()
)

In [1]:
retriever.invoke("한효주")

NameError: name 'retriever' is not defined

full_chain.invoke({"question": "한효주가 나오는 드라마 알려줘."})

# bori

In [17]:
def process_user_input(classification_result: dict):
    # 사용자의 입력 유형 분류
    print(classification_result)
    classification_data = classification_result.get("classification_result", {})  # 내부 딕셔너리 추출
    type_value = classification_data.get("type", "일반대화")  # 기본값 설정
    keywords = classification_data.get("keywords", [])  # 기본값 설정
    
    print(f"===================== Type: {type_value}")
    print(f"===================== Keywords: {keywords}")

    if type_value == '정보검색':
        return info_chain.invoke(str(classification_result))
    elif type_value == '추천요청':
        return "추천요청 체인 실행은 여기!!"
    else:
        return default_chain.invoke({"classification_result": classification_result})

In [18]:
full_chain = (
  {"classification_result": RunnableLambda(preprocess_classification_result),# type keyword: dic
   "question":itemgetter("question")}
  | RunnableLambda(process_user_input)
  | StrOutputParser()  
)

In [32]:
full_chain.invoke({"question":"더 이퀄라이저 시리즈가 몇개야 내놔"})

{'classification_result': {'type': '정보검색', 'keywords': ['더 이퀄라이저', '시리즈', '개수']}, 'question': '더 이퀄라이저 시리즈가 몇개야 내놔'}
===================== Type: 정보검색
===================== Keywords: ['더 이퀄라이저', '시리즈', '개수']


'{\n  "documents": [\n    {\n      "title": "더 이퀄라이저",\n      "genre": ["액션/어드벤쳐"]\n    },\n    {\n      "title": "더 이퀄라이저 2",\n      "genre": ["액션/어드벤쳐"]\n    },\n    {\n      "title": "더 이퀄라이저 3",\n      "genre": ["액션/어드벤쳐"]\n    }\n  ]\n}'

In [ ]:
# '{
# "documents": [
#    {"title": "더 이퀄라이저", "genre": ["액션/어드벤쳐"]},
#    {"title": "더 이퀄라이저 2","genre": ["액션/어드벤쳐"]},
#    {"title": "더 이퀄라이저 3","genre": ["액션/어드벤쳐"]}
#    ]
# }'

3.8666666666666667

In [42]:
6756/800

8.445

In [28]:
pd.read_csv("C:/Users/USER/Downloads/영화_TMDB_5800_Mapping-최종(250107).csv")

,name,orgnl_cntry,movie_id,overview,release_date,adult,backdrop_path,original_language,original_title,poster_path,popularity,runtime,vote_average,vote_count,genre,test
0,노크 노크,미국,263472,가족에게 헌신적이고 누구보다 아내와 아이들을 사랑하는 성공한 건축가 에반(키아누 리...,2015-06-26,False,/nJkp7hYKScmvQuqfptIIclFxZ94.jpg,en,Knock Knock,/8hjIz6BkZiBSeuHYs1e8eO0w489.jpg,61.593,99,5.368,3203,"공포, 스릴러",1
1,노크: 더 하우스,미국,709631,"이제 막 여덟 살이 된 소년 ‘피터’ . 밤이 찾아오면, 방에서 의문의 노크 소리가...",2023-07-19,False,/nYDPmxvl0if5vHBBp7pDYGkTFc7.jpg,en,Cobweb,/5Ol63pZT8JfYgFMfADqySpgwkdj.jpg,36.104,88,6.509,876,공포,1
2,노크,대한민국,255250,미대생 정화(서우)는 학교 근처의 작은 아파트로 이사를 하던중 순간적으로 발을 헛디...,2012-11-29,False,/4etMq6yuYyzzARECneGst6l1RWr.jpg,ko,노크,/bGgrseFG2FZd7dIhBmsFupEhlyt.jpg,0.259,100,3.700,3,"공포, TV 영화, 로맨스",2
3,한밤의 황당한 저주,미국,398798,야간 경비로 첫 출근을 하게 된 저스틴은 선임들의 고약한 인터뷰를 무사히 마치고 전...,2017-03-04,False,/r0HMSRMOCSXZSj6fcN7CXdJLtSk.jpg,en,The Night Watchmen,/b0kxlzG2mkJ3ljWOEtZ95Knpw4y.jpg,7.981,90,5.100,126,"코미디, 공포",1
4,신세계,대한민국,165213,경찰청 수사기획과 강과장은 국내 최대 범죄 조직 골드문이 기업형 조직으로 세력이 확...,2013-02-21,False,/8OceZqoCwVoc3JMN7ffsaj4BGFM.jpg,ko,신세계,/yiDQaLGF6hXVdiKodfdurUcgxWV.jpg,16.436,134,7.400,461,"스릴러, 범죄, 드라마",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5795,샤프트,미국,479,돈 많은 아버지를 둔 월터 웨이드(크리스찬 베일)는 사소한 시비 끝에 젊은 흑인 학...,2000-06-15,False,/sblTwYXpm6Nrl4qewg3526lBdlK.jpg,en,Shaft,/5ncvnNPaN73cOqD4muE413sSOyY.jpg,22.547,99,6.019,1273,"액션, 모험, 범죄, 스릴러",1
5796,페이첵,미국,9620,"가까운 미래, 천재 공학자 마이클 제닝스는 각종 회사의 일급 프로젝트를 수행하고 있...",2003-12-25,False,/rObU664nB6WmWUduCcwYOUYkPAt.jpg,en,Paycheck,/a9DUh3KJlxTHgngtVcCwPAfgknS.jpg,16.652,119,6.200,1808,"액션, 모험, 미스터리, SF, 스릴러",1
5797,아미스타드,미국,11831,"칠흑 같이 어두운 밤, 흑인들의 지도자격인 신케이는 쇠고랑을 푸는 데 성공하고 아미...",1997-12-10,False,/2lI5rNJeYHShg9okpEXSFzQZqWj.jpg,en,Amistad,/6QqNyIHKow0jngiQgTNBOBrLILM.jpg,4.085,155,7.026,1292,"드라마, 역사, 미스터리",1
5798,매니악,미국,27346,프랭크(조 스피넬 분)는 몇 년전 교통사고로 죽은 어머니를 항상 그리워한다. 매춘부...,1980-05-10,False,/h5X5zO40G0pjPkyyzGh9PBsPI0B.jpg,en,Maniac,/aKIBqbUZAEwOML4oMMjZS7m4Gvj.jpg,12.009,88,6.457,440,공포,1
